# QSDC with quantum one-time-pad

```
@misc{deng2022repeatable,
      title={Repeatable classical one-time-pad crypto-system with quantum mechanics}, 
      author={Fu-Guo Deng and Gui Lu Long},
      year={2022},
      eprint={1902.04218},
      archivePrefix={arXiv},
      primaryClass={quant-ph}
}
```

1. Bob prepares a sequence 𝑄 of single-qubit states, where each 𝑄𝑖 is randomly chosen from {|0⟩ , |1⟩ , |+⟩ , |−⟩} and sends the sequence 𝑄 to Alice.

2. Alice chooses some qubits of 𝑄 and randomly measures in 𝑍 and 𝑋 bases. Then they publicly discuss the positions and the measurement results to calculate the error rate in the channel. A higher error rate implies the presence of some eavesdroppers and then they abort. Otherwise, Alice applies the unitary 𝑈0 and 𝑈3, on the remaining qubits, to encode her message bit 0 and 1 respectively. Note that 𝑈0 does not change the states and 𝑈3 flips the states, i.e., 𝑈3 |0⟩ = |1⟩, 𝑈3 |1⟩ = |0⟩, 𝑈3 |+⟩ = |−⟩ and 𝑈3 |−⟩ = |+⟩.

3. Alice sends those encoded qubits to Bob, who measures those qubits in proper bases and
gets the secret message.

### Assumptions:

1. Channel is noiseless

In [239]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.tools.visualization import plot_histogram

In [240]:
simulator = Aer.get_backend('qasm_simulator')

In [241]:
def performMeasurement(circuit, qubit, basis):
	'''
	basis = C (Computational), H (Hadamard)
	'''
	if basis == 'C':
			circuit.measure(qubit, qubit)
	if basis == 'H':
			circuit.h(qubit)
			circuit.measure(qubit, qubit)
	return circuit

In [242]:
# bob sends bits

bobEncodedBits = ["0", "1", "+", "-", "0", "1", "+", "-"]

def bobEncodingCircuit(E):
    circuit = QuantumCircuit(1, 1)

    if E == "1":
        circuit.x(0)
    if E == "+":
        circuit.h(0)
    if E == "-":
        circuit.x(0)
        circuit.h(0)
    
    return circuit

bobCircuits = [bobEncodingCircuit(b) for b in bobEncodedBits]

In [243]:
# send/tamper

T = False # varible to decide if tampering should be performed
# T = True # varible to decide if tampering should be performed

# C = computational
# H = hadamard

tamperBases = ['H', 'C', 'H', 'C', 'H', 'H', 'C', 'C']

if T:
    for i in range(len(bobCircuits)):
        bobCircuits[i] = performMeasurement(bobCircuits[i], 0, tamperBases[i])

In [244]:
# alice measures first 4 bits

aliceVerificationCircuits = [None]*4
aliceVerificationBasis = ["C", "C", "H", "C"]

verificationResult = [None]*4
verificationCounts = [None]*4

for i in range(4):
	aliceVerificationCircuits[i] = performMeasurement(bobCircuits[i], 0, aliceVerificationBasis[i])
	verificationResult[i] = execute(aliceVerificationCircuits[i], backend=simulator).result()
	verificationCounts[i] = verificationResult[i].get_counts(aliceVerificationCircuits[i])

In [245]:
# verify

total = 4
correct = 0

for i in range(4):
    if len(list(verificationCounts[i].keys())) == 1: # i.e. basis and measurement basis match
        if aliceVerificationBasis[i] == 'C':
            if (bobEncodedBits[i] == list(verificationCounts[i].keys())[0] == "0") or (bobEncodedBits[i] == list(verificationCounts[i].keys())[0] == "1"):
                correct += 1
        else:
            if (bobEncodedBits[i] == "+" and list(verificationCounts[i].keys())[0] == "0") or (bobEncodedBits[i] == "-" and list(verificationCounts[i].keys())[0] == "1"):
                correct += 1

In [246]:
if correct/total < 0.5:
    print("Eavesdropper present")
else:
    print("Channel is secure")

    # encoding by alice

    # application of U0 = I => 0
    # application of U3 = iY = XZ => 1

    message = ["0", "1", "0", "1"]

    for i in range(4, 8):
        if message[i-4] == "1":
            bobCircuits[i].z(0)
            bobCircuits[i].x(0)

    # decoding by bob

    decodingResult = [None]*4
    decodingCounts = [None]*4

    decodedMessage = []

    for i in range(4, 8):
        if bobEncodedBits[i] == "0" or bobEncodedBits[i] == "1":
            bobCircuits[i] = performMeasurement(bobCircuits[i], 0, 'C')
            decodingResult[i-4] = execute(bobCircuits[i], backend=simulator).result()
            decodingCounts[i-4] = decodingResult[i-4].get_counts(bobCircuits[i])

            if bobEncodedBits[i] == list(decodingCounts[i-4].keys())[0]:
                decodedMessage.append("0")
            else:
                decodedMessage.append("1")
        else:
            bobCircuits[i] = performMeasurement(bobCircuits[i], 0, 'H')
            decodingResult[i-4] = execute(bobCircuits[i], backend=simulator).result()
            decodingCounts[i-4] = decodingResult[i-4].get_counts(bobCircuits[i])

            if (bobEncodedBits[i] == "+" and list(decodingCounts[i-4].keys())[0] == "0") or (bobEncodedBits[i] == "-" and list(decodingCounts[i-4].keys())[0] == "1"):
                decodedMessage.append("0")
            else:
                decodedMessage.append("1")
        # print(decodingCounts[i-4])
    print("Decoded message is:")
    print(" | ".join(decodedMessage))

Channel is secure
Decoded message is:
0 | 1 | 0 | 1
